# Configure Scan & System Health Displays Test

## Table of Contents
#### [1 - Global Configuration](#1-global-configuration)
#### [2 - Test Equipment Setup](#2-test-equipment-setup)
#### [3 - Configure Scan Tests](#3-configure-scan-tests)
#### [4 - Debug Telescope](#3-debug-telescope)

## 1 Global Configuration

### 1.1 Import dependencies

In [ ]:
import sys

sys.path.append("../../src")

import json
import logging
import os
import pathlib
import time
import ska_ser_logging
from bokeh.io import output_notebook
from ska_oso_pdm.entities.common.target import (
    CrossScanParameters,
    FivePointParameters,
    RasterParameters,
    SinglePointParameters,
    StarRasterParameters,
)
from ska_oso_pdm.entities.sdp import BeamMapping
from ska_oso_scripting import oda_helper
from ska_oso_scripting.functions.devicecontrol.resource_control import get_request_json
from ska_oso_scripting.objects import SubArray, Telescope
from ska_tmc_cdm.messages.central_node.assign_resources import AssignResourcesRequest
from ska_tmc_cdm.messages.central_node.sdp import Channel
from ska_tmc_cdm.messages.subarray_node.configure import ConfigureRequest
from ska_tmc_cdm.messages.subarray_node.configure.core import ReceiverBand

from ska_mid_jupyter_notebooks.cluster.cluster import Environment, TangoDeployment
from ska_mid_jupyter_notebooks.dish.dish import TangoDishDeployment
from ska_mid_jupyter_notebooks.helpers.path import project_root
from ska_mid_jupyter_notebooks.obsconfig.config import ObservationSB
from ska_mid_jupyter_notebooks.obsconfig.target_spec import TargetSpec
from ska_mid_jupyter_notebooks.sut.rendering import TelescopeMononitorPlot
from ska_mid_jupyter_notebooks.sut.state import TelescopeDeviceModel, get_telescope_state
from ska_mid_jupyter_notebooks.sut.sut import TangoSUTDeployment, disable_qa
from ska_mid_jupyter_notebooks.test_equipment.rendering import get_test_equipment_monitor_plot
from ska_mid_jupyter_notebooks.test_equipment.state import get_equipment_model
from ska_mid_jupyter_notebooks.test_equipment.test_equipment import TangoTestEquipment

### 1.2 Setup Global Variables and Configuration

In [ ]:
debug_mode = True  # This setting enables printing of diagnostics
enable_logging = True  # This enables logging and sets the global log_level to debug
dishlmc_enabled = True  # Set this to true if you have a dish LMC deployment
executon_environment = Environment.CI
branch_name = (
    "at-1952-comm-test"  # Set this if you are using an on-demand deployment (i.e. Environment.CI)
)
if enable_logging:
    ska_ser_logging.configure_logging(logging.DEBUG)
test_equipment = TangoTestEquipment()
print(f"Test Equipment Configured: {test_equipment}")
# namespace_override parameter can be used to override auto-configured SUT namespace
sut_namespace_override = ""
subarray_count = 1
subarray_id = 1
sut = TangoSUTDeployment(
    branch_name,
    executon_environment,
    namespace_override=sut_namespace_override,
    subarray_index=subarray_id,
)
print(f"SUT configured: {str(sut)}")
dish_ids = ["0001", "0036"]
# namespace_override parameter can be used to override auto-configured dish namespace
dish_namespace_overrides = ["", ""]
if dishlmc_enabled:
    dish_deployments = []
    for i, d in enumerate(dish_ids):
        dish = TangoDishDeployment(
            f"ska{d[1:]}",
            branch_name=branch_name,
            environment=executon_environment,
            namespace_override=dish_namespace_overrides[i],
        )
        print(f"Dish {d} configured: {dish}")
        dish_deployments.append(dish)
else:
    dish_deployments = []

timestr = time.strftime("%Y%m%d-%H%M")
notebook_output_dir = pathlib.Path(
    project_root(), f"notebook-execution-data/configure_scan_for_commissioning/execution-{timestr}"
)
os.makedirs(notebook_output_dir)
# we disable qa as it is not been properly verified
disable_qa()

### 1.3 Test Connections to Namespaces

In [ ]:
sut.smoke_test()
test_equipment.smoke_test()
for dish_deployment in dish_deployments:
    dish_deployment.smoke_test()

### 1.4 Export System Configuration

In [ ]:
deployment: TangoDeployment
for deployment in [sut, test_equipment, *dish_deployments]:
    deployment.export_chart_configuration(output_dir=notebook_output_dir)

## 2 Test Equipment Setup

Use the noise source at nominal levels for input to at least one SPFRx.  

### 2.1 Configure Test Equipment State

In [ ]:
test_equipment_state = get_equipment_model(test_equipment)
test_equipment.devices

### 2.2 Print Test Equipment Diagnostics

In [ ]:
test_equipment.print_diagnostics()

### 2.4 Turn offline Test Equipment devices ONLINE

In [ ]:
# set any offline devices to online
test_equipment.turn_online()

### 2.5 Display Test Equipment Device States 

In [ ]:
test_equipment_state.state["devices_states"]

### 2.6 Configure Signal Generator and set noise

In [ ]:
frequency_to_set = 800e6

signal_generator = test_equipment.signal_generator
print(f"Current signal generator frequency: {signal_generator.frequency}")
signal_generator.write_attribute("frequency", frequency_to_set)
time.sleep(1)
print(f"Updated signal generator frequency: {signal_generator.frequency}")
assert (
    signal_generator.frequency == frequency_to_set
), f"Frequency required is {frequency_to_set} but got {signal_generator.frequency}"

## 3 Configure Scan Tests

Use OSO scripting interface to TMC to run the script for assigning resources for a single sub-array, configuring and running a  scan.  

### 3.1 Setup Telescope Monitoring

#### 3.1.1 Configure Telescope Monitoring

In [ ]:
# setup monitoring
# use telescope state object for state monitoring
device_model = TelescopeDeviceModel(dish_ids, subarray_count)
telescope_state = get_telescope_state(device_model, sut)
# use monitor plot as a dashboard
telescope_monitor_plot = TelescopeMononitorPlot(plot_width=900, plot_height=200)
# set up events to monitor
telescope_state.subscribe_to_on_off(telescope_monitor_plot.observe_telescope_on_off)
telescope_state.subscribe_to_subarray_resource_state(
    telescope_monitor_plot.observe_subarray_resources_state
)
telescope_state.subscribe_to_subarray_configurational_state(
    telescope_monitor_plot.observe_subarray_configuration_state
)
telescope_state.subscribe_to_subarray_scanning_state(
    telescope_monitor_plot.observe_subarray_scanning_state
)
output_notebook()

#### 3.1.2 Open the inline dashboard
Start the simple inline dashboard showing current state of the Telescope and resource assignment and configuration status.

In [ ]:
telescope_monitor_plot.show()
telescope_state.activate()
telescope_state.wait_til_ready(2)

#### 3.2.3 Print CBF Diagnostics

In [ ]:
sut.print_cbf_diagnostics()

#### 3.2.4 Print SDP Diagnostics

In [ ]:
sut.print_sdp_diagnostics()

#### 3.2.5 Print Dish-LMC Diagnostics

In [ ]:
for dish_deployment in dish_deployments:
    print(f"Dish {dish_deployment.dish_id} - {dish_deployment.namespace}: Diagnostics")

    dish_deployment.print_diagnostics()

#### 3.2.6 Print Full System Diagnostics

In [ ]:
print("SUT: Full Diagnostics")
sut.print_full_diagnostics()

for dish_deployment in dish_deployments:
    print(f"Dish {dish_deployment.dish_id}: Full Diagnostics")
    dish_deployment.print_full_diagnostics()

print("Test Equipment: Full Diagnostics")
test_equipment.print_full_diagnostics()

### 3.4 Initialise Telescope and Subarray
Create Subarray and Telescope instances.

In [ ]:
sub = SubArray(subarray_id)
tel = Telescope()

### 3.5 Load Dish-VCC Configuration in TMC

In [ ]:
# This should only be executed for a fresh deployment (i.e. Telescope is OFF.
# If you have restarted the subarray, you should not run this command
sut.load_dish_vcc_config()

### 3.6 Turn telescope ON

In [ ]:
# set to ON only if OFF
# If you have restarted the subarray, you should not run this command (Telescope is already ON)
# dish_lmc mode must be in LP_standby and before trying to turn the telescope ON
# Takes about 1m20s
tel.on()

### 3.7 Change Telescope to Operate Mode

In [ ]:
for dish_deployment in dish_deployments:
    vars(dish_deployment)
    dish_deployment.set_dish_operational()

## 4 Debug Telescope

### 4.1 Display Telescope State

In [ ]:
telescope_state.state

### 4.2 Display Telescope State Subscriptions

In [ ]:
telescope_state.state_monitor.subscriptions

### 4.3 Display Dish LMC State

In [ ]:
dishes_to_debug = [d.dish_id for d in dish_deployments]
for dish in dish_deployments:
    if dish.dish_id in dishes_to_debug:
        dish.print_diagnostics()

### 4.4 Display CSP LMC State

In [ ]:
sut.print_csp_diagnostics()

In [ ]:
sut.print_full_diagnostics()

### 4.5 Display TMC State

In [ ]:
sut.print_tmc_diagnostics()

### 4.6 Display SDP State

In [ ]:
sut.print_sdp_diagnostics()

## Control and Monitoring Communications Commissioning Test

### Copied from Jama SKAO-TC-2741 (10/04/2024)